# Curso de aprendizaje automatizado
PCIC, UNAM

Machine Learning

Rodrigo S. Cortez Madrigal

<img src="https://pcic.posgrado.unam.mx/wp-content/uploads/Ciencia-e-Ingenieria-de-la-Computacion_color.png" alt="Logo PCIC" width="128" />   

### Tarea 4

Calcula la representación de bolsa de palabras con tf-idf de la base de datos de 20 newsgroups y
realiza lo siguiente:

- 1. Compara el rendimiento de la SVM con kernel coseno, lineal y RBF.
- 2. Prueba que el kernel coseno es Mercer:

In [20]:
import spacy
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

In [10]:
from sklearn.datasets import fetch_20newsgroups

newsgroups = fetch_20newsgroups(subset='all', data_home='./data')

In [24]:
# Transform the data into a DataFrame
df = pd.DataFrame(newsgroups.data, columns=['text'])
df['target'] = newsgroups.target
df['target_names'] = df['target'].progress_apply(lambda x: newsgroups.target_names[x])
df['target_names'] = df['target_names'].progress_apply(lambda x: x.replace('_', ' '))

100%|██████████| 18846/18846 [00:00<00:00, 2519791.30it/s]


In [17]:
df

,text,target,target_names
0,From: Mamatha Devineni Ratnam <mr47+@andrew.cm...,10,rec.sport.hockey
1,From: mblawson@midway.ecn.uoknor.edu (Matthew ...,3,comp.sys.ibm.pc.hardware
2,From: hilmi-er@dsv.su.se (Hilmi Eren)\nSubject...,17,talk.politics.mideast
3,From: guyd@austin.ibm.com (Guy Dawson)\nSubjec...,3,comp.sys.ibm.pc.hardware
4,From: Alexander Samuel McDiarmid <am2o+@andrew...,4,comp.sys.mac.hardware
...,...,...,...
18841,From: jim.zisfein@factory.com (Jim Zisfein) \n...,13,sci.med
18842,From: rdell@cbnewsf.cb.att.com (richard.b.dell...,12,sci.electronics
18843,From: westes@netcom.com (Will Estes)\nSubject:...,3,comp.sys.ibm.pc.hardware
18844,From: steve@hcrlgw (Steven Collins)\nSubject: ...,1,comp.graphics


In [ ]:
nlp = spacy.load("en_core_web_sm")

def TokenizeText(text, nlp):
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return tokens

df['text'] = df['text'].progress_apply(lambda x: TokenizeText(x, nlp))

In [ ]:
texts = df['text'].progress_apply(lambda x: ' '.join(x))

# Crear la matriz TF
tf = TfidfVectorizer(use_idf=False, norm='l2')
tf_matrix = tf.fit_transform(texts)
tf_feature_names = tf.get_feature_names_out()

# Crear la matriz TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(texts)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()

# Crear matriz BOW
bow_vectorizer = TfidfVectorizer(use_idf=False, norm=None)
bow_matrix = bow_vectorizer.fit_transform(texts)
bow_feature_names = bow_vectorizer.get_feature_names_out()

100%|██████████| 18846/18846 [00:00<00:00, 248467.17it/s]


## 1. Compara el rendimiento de la SVM con kernel coseno, lineal y RBF.

In [34]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
import numpy as np

# Definir los kernels
kernels = {
    'lineal': 'linear',
    'RBF': 'rbf',
    'coseno': lambda X, Y: cosine_similarity(X, Y)
}

# Evaluar cada kernel
results = {}
for name, kernel in kernels.items():
    if name == 'coseno':
        svc = SVC(kernel='precomputed')
        kernel_matrix = cosine_similarity(tfidf_matrix)
        scores = cross_val_score(svc, kernel_matrix, df['target'], cv=5)
    else:
        svc = SVC(kernel=kernel)
        scores = cross_val_score(svc, tfidf_matrix, df['target'], cv=5)
    
    results[name] = {
        'mean_accuracy': np.mean(scores),
        'std_accuracy': np.std(scores)
    }

# Mostrar resultados
for kernel, metrics in results.items():
    print(f"Kernel: {kernel}")
    print(f"Mean Accuracy: {metrics['mean_accuracy']:.4f}")
    print(f"Std Accuracy: {metrics['std_accuracy']:.4f}")

Kernel: lineal
  Mean Accuracy: 0.9189
  Std Accuracy: 0.0040
Kernel: RBF
  Mean Accuracy: 0.9113
  Std Accuracy: 0.0052
Kernel: coseno
  Mean Accuracy: 0.9189
  Std Accuracy: 0.0040
